## Python Code Integration (aka., How to insert python into other code processes or visa-versa.)
Adam Shaver July 15th, 2015

### ToolChain
- Clean Ubuntu 12.04 LTS (GCC 4.8.2, Python 2.7.6)
 1.sudo apt-get install -y build-essential python-dev libbz2-dev
- [optional] Sublime
- Boost 1.58.0
 - Compiled from source, just good practice to make sure everything works on the system
As root
 - mkdir -p /usr/local/boost/1.58
 - boostrap.sh –prefix=/usr/local/boost/1.58
 - b2.sh -j<n> install
- Anaconda
- WinPDB 1.4.8 
 - conda install wxpython
 - python setup.py install -f
- Zato.io (see the topic for install info)
 
### Motivation
- Using Python for prototyping
- Using Python as middleware/glue between legacy code
- There are things that each language does well (C++, Fortran, Python) 
- Code Obfuscation
- Unit tests framework in one language, code in another
- If you're here, you probably have a reason

### Avenues of Integration
- Topic 1: Ctypes : Python --> C/C++ 
- Topic 2: pyObject : Python <--> C/C++
- Topic 3: The Boost Python C++ Library: C/C++ <--> Python
- Topic 4: ESB (Zato.io) Python <--> [Anything]
- Topic 5: [Not Covered] Other wrappers and translation layers, SIP, SWIG, Goggle Proto Buffer

*Note, there are plenty more.*

In [ ]:
from __future__ import print_function

## Topic 1: Ctypes, Python --> C/C++, using simple types from the C Language
Ctypes are defined mappings between C and Python.
- https://docs.python.org/2/library/ctypes.html
- https://docs.python.org/3/library/ctypes.html
- https://wakari.io/nb/url///wakari.io/static/notebooks/Lecture_6A_Fortran_and_C.ipynb


They serve to allow passing data to/from C/C++ functions that expect C-like data. Note, if the library expects some more complicated data structure/class you might be out of luck.

In [ ]:
import ctypes
curl_loc = ctypes.util.find_library('curl')
print(curl_loc)

In [ ]:
!locate libcurl

Examine all the objects. Then the exported objects. Note, if you write C++ code you wil need to prevent name mangling with the directive ``` extern "C" ``` 

In [ ]:
!nm -D /usr/lib/x86_64-linux-gnu/libcurl.so.4 

In [ ]:
!nm -D /usr/lib/x86_64-linux-gnu/libcurl.so.4 |grep "T "

---
We can load libraries in 4 ways using ctypes (CDLL, OleDLL, WinDLL, PyDLL). Items 2,3 are Windows only. You can use this method if the function signature is well defined, documented (e.g., can be formed with ctypes).

In [ ]:
curl = ctypes.CDLL(curl_loc)
curl_version_info = curl.curl_version_info
print(curl_version_info())
easy_handle = curl.curl_easy_init()

# This magic number came from a header typdef enum
curl.curl_easy_setopt(easy_handle, 0x100000+1 , "http://example.org") 

In [15]:
%%file code/hello.cpp
#include <iostream>
#include <string.h>
using namespace std;
extern "C" {
    int main () {
        cout << "Hello Main2\n";
        return 42;
    }
    
    string helloStr() {
        return "Hello Str\n";
    }
    
    const char* helloStrPtr() {
        char* p = new char[19];
        strcpy(p,"Hello Str Pointer\n");
        return p;
    }
    
    void helloStrRef(char* p) {
        strcpy(p,"Hello Str Ref!\n");
    }
    
    struct MsgWrapper {
        char * msgIn = new char[46];
        char * msgOut = new char[46];
    };
    
    void helloStrRefFromStruct(MsgWrapper s) {
        strcpy(s.msgIn, "Aloha\n");
    }
    
    int missing_answer(int  i) {
        i = (42-i);
        return i;
    }
    
     
}

Overwriting code/hello.cpp


Compile to a shared library. Note, if you are on Windows you'll need to declare an export symbol or use an export library. In Linux, because all dynamic libraries share a heap, all the symbols are exported. 

In [16]:
! rm bin/hello bin/libHello.so
!g++ -c -std=c++11 -fPIC code/hello.cpp -o code/hello.o
!g++ -o bin/libHello.so -shared code/hello.o
!g++ -o bin/hello code/hello.o
!bin/hello

Hello Main2


In [93]:
!nm -D bin/libHello.so |grep 'T '

0000000000000ea0 T _fini
0000000000000d87 T hello_obj_factory
0000000000000cb6 T helloStr
0000000000000d19 T helloStrPtr
0000000000000d5e T helloStrRef
0000000000000db1 T helloStrRefFromObj
0000000000000ab0 T _init
0000000000000c95 T main


###Executing a function. 
(1) As an aside, I routinely import ctypes in each cell and reload the library, because if anything goes wrong, the kernel crashes (and the dynamic loaded library is lost). (2) Also, I explicitly didn't import * from cTypes so you can see which functions are from the module.

In [35]:
import ctypes
helloCpp = ctypes.CDLL('./bin/libHello.so')
print(helloCpp.main()) # Note this prints the return code, not hello
print(helloCpp.helloStr()) # Note this doesn't wactually pass back a string, but the pointer to a string.

# These will attempt to deref a pointer to a value that may not exist.
#a_str_ptr = helloCpp.helloStr()
#print(ctypes.c_char_p(a_str_ptr).value)


42
-121235056


Executing a function and catching the result. Note, it is pointer to a string, so we will have to dereference it to get the string value. 

In [4]:
import ctypes
helloCpp = ctypes.CDLL('./bin/libHello.so')
a_str_ptr = helloCpp.helloStrPtr() # Returns a pointer to a string (char* or c_char_p)
print(a_str_ptr)
#print(a_str_ptr.value) Fails to deref as python doesn't have it cast as a cType.c_char_p.
print(ctypes.c_char_p(a_str_ptr).value) # Dereferences it to get the char

28594528
Hello Str Pointer



We can also pass a reference to a string into the code.

In [41]:
import ctypes
helloCpp = ctypes.CDLL('./bin/libHello.so')
a_str_ptr2 = ctypes.create_string_buffer('\000' * 45)
helloCpp.helloStrRef(ctypes.byref(a_str_ptr2)) # passes in the pointer 
print(a_str_ptr2)
print(a_str_ptr2.value) 

Hello Str Ref!



We can even generate structs and such to pass the data back and forth

In [1]:
import ctypes
helloCpp = ctypes.CDLL('./bin/libHello.so')
class MsgWrapper(ctypes.Structure):
    _fields_ = [("msgOut", ctypes.c_char_p), ("msgIn", ctypes.c_char_p)]
amsg = MsgWrapper("Goodbye", "Hello")
print(amsg.msgIn)
print(amsg.msgOut)

Hello
Goodbye


In [2]:
helloCpp.helloStrRefFromStruct(amsg)
print(amsg.msgIn)
print(amsg.msgOut)

Hello
No Seriously, Bye



In [17]:
import ctypes
helloCpp = ctypes.CDLL('./bin/libHello.so')
print(helloCpp.missing_answer(24))

18


###Pitfalls with ctypes
----
 - Endian specification
 - C++ name mangling
 - memory leaks (allocation in c/c++ without deallocation). These can easily happen across the interface if you're not passing by reference (with byref).
 - pointer problems 

What about more complicated input and output? C-types only have structs and unions, but not classes. Therefore, we can't pass around objects from a C++ library. Enter pyObject......

##Topic 2: pyObject,  Python in modules C/C++ 

A pyObject is a Python object in C.  Also, *everything* in Python is really a pyObject. Using it, we get inerfaces that can do anything python can do (though the wrapping of complicated Python objects requires equally complicated code). Strangely, everything I've seen uses pyObjects to make modules, rather than some lightweight interface like ctypes. 
 - https://docs.python.org/2/extending/extending.html
 - https://docs.python.org/3/extending/extending.html
 - http://intermediate-and-advanced-software-carpentry.readthedocs.org/en/latest/c++-wrapping.html
 - (my favorite, includes numpy headers, deploying with setup.py, etc) http://dan.iel.fm/posts/python-c-extensions/
 
To start, what do you need in a module containing :
  1. Function: The function which takes and possibly returns a pyObject
  1. Method Table: A static definition of the function signature, how it is called, it's name in python, in C/C++
  1. Initialization: An initialization call for the function (which I belive registers the module and this function as a function of the module.

In [19]:
%%file code/Simple.cpp
#include <python2.7/Python.h>  //Contains the spec for the Python API, pyObject, etc.

//Function 1
static PyObject * answer_to_everything(PyObject *self, PyObject *args)
{
    return PyLong_FromLong(42); // convert to Python object of a type
}

//Function 2
static PyObject * missing_from_answer_to_everything(PyObject *self, PyObject *args)
{
    int sts;
    if (!PyArg_ParseTuple(args, "i", &sts)) //"((ii)(ii))(ii)", is f((0, 0), (400, 300)), (10, 10))
        return NULL;                        //"s|si", is f('wer','d', 10) or f('cmd')
    sts = 42 - sts;
    PyObject * ret = PyLong_FromLong(sts); // convert to Python object of a type
    return ret;
}

// Static that maps C names to python and outlines the variables
static PyMethodDef SimpleMethods[] = {
    {"what_is_the_answer",  answer_to_everything, METH_VARARGS,
     "Gets The Answer."}, 
    {"missing_answer",  missing_from_answer_to_everything, METH_VARARGS,
     "Finds what you're missing."},
    {NULL, NULL, 0, NULL}        /* Sentinel */
};



// Registration of the functions and generation of the module
PyMODINIT_FUNC
initSimple(void)  // MUST be named "initFoo" for a library with name of "Foo"
{
    (void) Py_InitModule("Simple", SimpleMethods);
}

Overwriting code/Simple.cpp


When compiling this, you have to explicitly link in the python library. Otherwise, the objects will compile, but the functions for things like (PyArg_ParseTuple won't be defined).
Also, note that the PyMODINIT_FUNC makes the init an "extern C" function without mangling.

In [20]:
! rm bin/Simple.so Simple.so
!g++ -c -std=c++11 -fPIC code/Simple.cpp -o code/Simple.o 
!g++ -o bin/Simple.so -shared code/Simple.o -L/usr/lib -lpython2.7
!nm -D bin/Simple.so |grep 'T '
!ln -s ./bin/Simple.so Simple.so

000000000000086c T _fini
0000000000000658 T _init
0000000000000843 T initSimple


In [4]:
import Simple
print(Simple.what_is_the_answer(234234))
print(Simple.missing_answer(23))


42
19


If we want to be clever, we could pass a function into C/C++. 

In [2]:
%%file code/SimpleFcn.cpp
#include <python2.7/Python.h>  //Contains the spec for the Python API, pyObject, etc.

// First we need to accept and set Python function as the callback
// Note, this set function is right from the Python Documentation
static PyObject *my_callback = NULL;
static PyObject * my_set_callback(PyObject *dummy, PyObject *args)
{
    PyObject *result = NULL;
    PyObject *temp;

    if (PyArg_ParseTuple(args, "O:set_callback", &temp)) {
        if (!PyCallable_Check(temp)) {
            PyErr_SetString(PyExc_TypeError, "parameter must be callable");
            return NULL;
        }
        Py_XINCREF(temp);         /* Add a reference to new callback */
        Py_XDECREF(my_callback);  /* Dispose of previous callback */
        my_callback = temp;       /* Remember new callback */
        /* Boilerplate to return "None" */
        Py_INCREF(Py_None);
        result = Py_None;
    }
    return result;
}


//Now we call the funciton
static PyObject * simple_callback(PyObject *self, PyObject *args)
{
    return PyObject_CallObject(my_callback, args);
}

//Calling a more pedantic fashion to highlight decrementing for garbage collection
//Also, we could do something with the result in C/C++ if we wanted
//Note, modified from  Python Documetation
static PyObject * pedantic_callback(PyObject *self, PyObject *args)
{
    double val_in;
    if (!PyArg_ParseTuple(args, "d", &val_in)) 
        return NULL;  
    //val_in = val_in*42;
    //return PyFloat_FromDouble(val_in);
    
    PyObject *arglist;
    PyObject *result;
    arglist = Py_BuildValue("(d)", val_in);
    //return arglist; //Actually breaks from Python and returns (foo,) rather than (foo)
    result = PyObject_CallObject(my_callback, arglist);
    Py_DECREF(arglist); // Note decref to release args. It was only needed for the fcn call.
    //return arglist; // Would return ((<NULL>,),)
    return result;
    
}

// Static that maps C names to python and outlines the variables
static PyMethodDef SimpleMethods[] = {
    {"set_fcn_callback",  my_set_callback, METH_VARARGS,
     "Sets_a_callback."},
    {"simple_callback",  simple_callback, METH_VARARGS,
     "calls back with passthrough args."},
    {"pedantic_callback",  pedantic_callback, METH_VARARGS,
     "calls back with explicit int args."},
    {NULL, NULL, 0, NULL}        /* Sentinel */
};

// Registration of the functions and generation of the module
PyMODINIT_FUNC
initSimpleFcn(void)  // MUST be named "initFoo" for a library with name of "Foo"
{
    (void) Py_InitModule("SimpleFcn", SimpleMethods);
}

Overwriting code/SimpleFcn.cpp


In [ ]:
%%file code/setup.py

from distutils.core import setup, Extension

setup(
    ext_modules=[Extension("SimpleFcn", ["SimpleFcn.cpp"])],
)


Go build this in conda's python via ```python setup.py build_ext --inplace```

In [3]:
! rm bin/SimpleFcn.so SimpleFcn.so
!g++ -c -std=c++11 -fPIC code/SimpleFcn.cpp -o code/SimpleFcn.o 
!g++ -o bin/SimpleFcn.so -shared code/SimpleFcn.o -L/usr/lib -lpython2.7
!nm -D bin/SimpleFcn.so |grep 'T '
!ln -s ./bin/SimpleFcn.so SimpleFcn.so

0000000000000b98 T _fini
00000000000007e8 T _init
0000000000000b6c T initSimpleFcn


In [ ]:
import SimpleFcn
def what_is_the_answer_with_inflation(rate):
    #%debug
    #import rpdb2; rpdb2.start_embedded_debugger('12345')
    return 42*rate
print(what_is_the_answer_with_inflation(1.02))   
SimpleFcn.set_fcn_callback(what_is_the_answer_with_inflation)
print(SimpleFcn.simple_callback(1.02))
print(SimpleFcn.simple_callback(1.07))
print(SimpleFcn.pedantic_callback(1.02))
print(SimpleFcn.pedantic_callback("1.07"))


###Pitfalls with pyObject
----
 - The *first* line of the C/C++ file should be '''#include < python2.7/Python.h\> ''' because it may alter how other headers are loaded (such as iostream). 
   - Note, most tutorials will have you just write '''< Python.h\>''', which won't find the file
   - This is available in Ubuntu via python-dev  
   - When linking, you need to include the python library libpython<ver>. In this case, it's libpython2.7.so.
 - Reference counting needs to be done correctly to prevent memory leaks when variables are generated on one side and destroyed on another. If the same variable is passed back/forth it
 - You can run into strange errors when you're not converting to the right Python type (e.g., long when int needed, etc).
   -This is all documented in the c-api documentation.

##Topic 3: Boost Python,  Seamless Python and Boost Integration

First, the example from boost (boost_1_58_0/libs/python/doc/index.html).
 - Note, I needed to put these libraries on the path "export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/boost/1.58/lib
"
 - If you notice, this the easiest implementation so far. 

In [2]:
%%file code/hello_ext.cpp
#include <boost/python.hpp>

char const* greet()
{
   return "hello, world";
}

BOOST_PYTHON_MODULE(hello_ext) //Will define the module init name via macros (see objects below)
{
    using namespace boost::python;
    def("greet", greet);
}


Overwriting code/hello_ext.cpp


In [3]:
!rm bin/hello_ext.so hello_ext.so
!g++ -c -std=c++11 -fPIC code/hello_ext.cpp -o code/hello_ext.o -I/home/ashaver/anaconda/include/python2.7 -I/usr/local/boost/1.58/include
!g++ -o bin/hello_ext.so -shared code/hello_ext.o /usr/local/boost/1.58/lib/libboost_python.so
!nm -D bin/hello_ext.so |grep 'T '
!ln -s ./bin/hello_ext.so hello_ext.so
!ldd hello_ext.so


0000000000005dc8 T _fini
0000000000004948 T _init
0000000000005002 T inithello_ext
000000000000501e T _Z21init_module_hello_extv
0000000000004ff5 T _Z5greetv
	linux-vdso.so.1 =>  (0x00007fffcaf72000)
	libboost_python.so.1.58.0 => /usr/local/boost/1.58/lib/libboost_python.so.1.58.0 (0x00007fd8c2299000)
	libstdc++.so.6 => /usr/lib/x86_64-linux-gnu/libstdc++.so.6 (0x00007fd8c1f7f000)
	libgcc_s.so.1 => /lib/x86_64-linux-gnu/libgcc_s.so.1 (0x00007fd8c1d68000)
	libc.so.6 => /lib/x86_64-linux-gnu/libc.so.6 (0x00007fd8c19a2000)
	libm.so.6 => /lib/x86_64-linux-gnu/libm.so.6 (0x00007fd8c169c000)
	/lib64/ld-linux-x86-64.so.2 (0x00007fd8c26f1000)


In [1]:
# Must have python on the path.
import hello_ext
print hello_ext.greet()

hello, world


Let's pass in a variable and a function, just like with ctypes and pyObject types. We're going to use the pattern on BOOST_PYTHON_FUNCTION_OVERLOADS (boost_1_58_0/libs/python/doc/tutorial/doc/html/python/functions.html)

In [1]:
%%file code/boost42.cpp
#include <boost/python.hpp>
        
int missing_answer(int a) {
    return (42-a);
}

BOOST_PYTHON_MODULE(boost42) //Will define the module init name via macros (see objects below)
{
    using namespace boost::python;
    def("missing_answer", missing_answer);
}



Overwriting code/boost42.cpp


In [14]:
!rm bin/boost42.so boost42.so
!g++ -c -std=c++11 -fPIC code/boost42.cpp -o code/boost42.o -I/home/ashaver/anaconda/include/python2.7 -I/usr/local/boost/1.58/include
!g++ -o bin/boost42.so -shared code/boost42.o /usr/local/boost/1.58/lib/libboost_python.so
!nm -D bin/boost42.so |grep 'T '
!ln -s ./bin/boost42.so boost42.so
!ldd boost42.so

rm: cannot remove ‘bin/boost42.so’: No such file or directory
0000000000006c20 T _fini
00000000000053b0 T _init
0000000000005b76 T initboost42
0000000000005b65 T _Z14missing_answeri
0000000000005b92 T _Z19init_module_boost42v
	linux-vdso.so.1 =>  (0x00007fff627fc000)
	libboost_python.so.1.58.0 => /usr/local/boost/1.58/lib/libboost_python.so.1.58.0 (0x00007fe4f8026000)
	libstdc++.so.6 => /usr/lib/x86_64-linux-gnu/libstdc++.so.6 (0x00007fe4f7d0c000)
	libgcc_s.so.1 => /lib/x86_64-linux-gnu/libgcc_s.so.1 (0x00007fe4f7af5000)
	libc.so.6 => /lib/x86_64-linux-gnu/libc.so.6 (0x00007fe4f772f000)
	libm.so.6 => /lib/x86_64-linux-gnu/libm.so.6 (0x00007fe4f7429000)
	/lib64/ld-linux-x86-64.so.2 (0x00007fe4f8480000)


We can call python from boost in various ways. This is called "embedding" in Boost lingo.
 - object eval(str expression, object globals = object(), object locals = object())
 - object exec(str code, object globals = object(), object locals = object())
 - object exec_file(str filename, object globals = object(), object locals = object())
 - or...import a module and work within its namespace

In [3]:
%%file code/boostUsingPython.cpp
#include <boost/python.hpp>
#include <iostream>
using namespace boost::python;
        
int main() {
    Py_Initialize();
    object random_module = import("random");
    object rand_namespace = random_module.attr("__dict__");
    object ignored = exec("result = gauss(2,5)", rand_namespace);
    double val = extract<double>(rand_namespace["result"]);
    std::cout << val <<'\n';
    return 0;
};

Overwriting code/boostUsingPython.cpp


In [5]:
!g++ -c -std=c++11 -fPIC code/boostUsingPython.cpp -o code/boostUsingPython.o -I/home/ashaver/anaconda/include/python2.7 -I/usr/local/boost/1.58/include
!g++ -o bin/boostUsingPython code/boostUsingPython.o /usr/local/boost/1.58/lib/libboost_python.so -lpython2.7
!bin/boostUsingPython
!bin/boostUsingPython
!bin/boostUsingPython
!bin/boostUsingPython
!bin/boostUsingPython


1.28748
5.99511
2.04232
3.06392
2.48043


###Pitfalls with Boost Python
----
 - Compiling boost, not having it on the path at runtime (which is easy to catch)
 - Not extracting the correct type
 - Something goes wrong in the macros, which you would have to read the C-header compiled code (before the object file is made)
 - Not putting the #include <boost/python.hpp> at the top
 - Forgetting to link in the boost python library 

## Topics 1-3 Wrap up
We compare the speed of the three interfaces below.
    - All three interface types are comperably quick (unless your bottleneck is actually the interface).
    - Boost has the smallest code, thought takes the longest to compile
    - The pyObject interface is the fastest, for whatever reason.
    - cTypes will sometimes let you work with native libraires, iff they take simple types.
I personally prefer boost, as it takes care of the references via smart pointers, extends well to complicated things, meshes will with C/C++ overloading conventions, etc.

In [8]:
import ctypes
helloCpp = ctypes.CDLL('./bin/libHello.so')
print(helloCpp.missing_answer(24)) 
%timeit helloCpp.missing_answer(24)
%time helloCpp.missing_answer(24)

import Simple
print(Simple.missing_answer(24))
%timeit Simple.missing_answer(24)
%time Simple.missing_answer(24)

import boost42
print(boost42.missing_answer(24))
%timeit boost42.missing_answer(24)
%time boost42.missing_answer(24)

18
The slowest run took 24.46 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 283 ns per loop
CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 16 µs
18
The slowest run took 24.20 times longer than the fastest. This could mean that an intermediate result is being cached 
10000000 loops, best of 3: 167 ns per loop
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 12.9 µs
18
The slowest run took 31.60 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 219 ns per loop
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 16.9 µs


18

##Topic 4: Zato.io, using an ESB to integrate Python

Installation/download is different for each platform:
- https://zato.io/downloads.html
- https://zato.io/downloads-2.0-ubuntu.html

My install instrutions (curl didn't work, substituted wget and cat). 

---

- sudo apt-get install apt-transport-https
- sudo apt-get install python-software-properties software-properties-common
- wget https://zato.io/repo/zato-0CBD7F72.pgp.asc
- cat zato-0CBD7F72.pgp.asc | sudo apt-key add -
- sudo apt-add-repository https://zato.io/repo/stable/2.0/ubuntu
- sudo apt-get update
- sudo apt-get -y install zato
- reboot, as the install made a user 'zato ESB'
---
  

Tutorial (From Zato)
 - Option 1, using Docker (https://zato.io/docs/2.0/admin/guide/install-docker.html)
 - Option 2
  - Getting redis (http://redis.io/)
    - wget http://download.redis.io/releases/redis-3.0.2.tar.gz
    - tar -xzvf redis-3.0.2.tar.gz
    - cd redis-3.0.2
    - make 
    - make test
    - sudo make install
    - cd utils
    - sudo ./install_server.sh
   
    ```
    Success!
    Starting Redis server...
    Installation successful! 
    ```
  - cd /usr/local/bin
  - sudo ln -s /opt/zato/2.0.5/bin/zato zato # Was missing line in the tutorial...zato wasn't in bin.
  - sudo su zato -
  - cd ~
  - mkdir -p env/qs-1
  - /opt/zato/2.0.5/bin/zato quickstart create ~/env/qs-1 sqlite localhost 6379 --kvdb_password '' --verbose
  - This output is **important** as it contains log-in info for administration!
 ```
 [1/8] Certificate authority created
[2/8] ODB schema created
[3/8] ODB initial data created
[4/8] server1 created
[5/8] server2 created
[6/8] Load-balancer created
Superuser created successfully.
[7/8] Web admin created
[8/8] Management scripts created
Quickstart cluster quickstart-578017 created
Web admin user:[admin], password:[ibes-fiar-esim-moye]
Start the cluster by issuing the /opt/zato/env/qs-1/zato-qs-start.sh command
Visit https://zato.io/support for more information and support options
```
  - /opt/zato/env/qs-1/zato-qs-start.sh
  ```
Starting the Zato cluster quickstart-578017
Running sanity checks
[1/6] Redis connection OK
[2/6] SQL ODB connection OK
[3/6] Load-balancer started
[4/6] server1 started
[5/6] server2 started
[6/6] Web admin started
Zato cluster quickstart-578017 started
Visit https://zato.io/support for more information and support options
```
  - zato check-config /opt/zato/env/qs-1/server1
  - zato check-config /opt/zato/env/qs-1/server2
  - curl localhost:11223/zato/ping ; echo
  

In [2]:
%%file zato_tutorial/my_service.py

from zato.server.service import Service

class GetClientDetails(Service):
    def handle(self): # The only method that *has* to be defined. This is the event-triggered entry point.
        self.log_input()

Writing zato_tutorial/my_service.py


In [1]:
!curl http://tutorial.zato.io/get-customer
!curl http://tutorial.zato.io/get-last-payment

{
 "firstName": "Sean", 
 "lastName": "O'Brien"
}
{
 "DATE": "2013-05-14T10:42:14.401555", 
 "AMOUNT": "357"
}


In [5]:
%%file zato_tutorial/my_service.py
from zato.server.service import Service

class GetClientDetails(Service):
    def handle(self): # The only method that *has* to be defined. This is the event-triggered entry point.
        self.logger.info('Request: {}'.format(self.request.payload))
        self.logger.info('Request type: {}'.format(type(self.request.payload)))

        # Fetch connection to CRM
        crm = self.outgoing.plain_http.get('CRM')

        # Fetch connection to Payments
        payments = self.outgoing.plain_http.get('Payments')

        # Grab the customer info ..
        response = crm.conn.send(self.cid, self.request.payload)
        cust = response.data

        # .. and last payment's details
        response = payments.conn.send(self.cid, self.request.payload)
        last_payment = response.data

        self.logger.info('Customer details: {}'.format(cust))
        self.logger.info('Last payment: {}'.format(last_payment))

        response = {}
        response['first_name'] = cust['firstName']
        response['last_name'] = cust['lastName']
        response['last_payment_date'] = last_payment['DATE']
        response['last_payment_amount'] = last_payment['AMOUNT']

        self.logger.info('Response: {}'.format(response))
        #import rpdb2; rpdb2.start_embedded_debugger('12345') # doesnt' work....
        
        # And return response to the caller
        self.response.payload = response

Overwriting zato_tutorial/my_service.py


In [6]:
!sudo su zato cp zato_tutorial/my_service.py /opt/zato/env/qs-1/server2/pickup-dir/

/bin/cp: /bin/cp: cannot execute binary file


In [ ]:
!curl localhost:11223/tutorial/first-service -d '{"cust_id":123, "cust_type":"A"}'
    

Go read and work through https://zato.io/docs/tutorial/02.html
Imho, worth it to get an introduction to zmq

In [ ]:
!ls /opt/zato/2.0.5/bin/

We're going to install our package Simple from earlier. 
 - cp simplePkg .
 - /opt/zato/2.0.5/bin/python setup.py build_ext install

In [5]:
%%file zato_tutorial/my_answer.py

from zato.server.service import Service
import Simple

class GetClientAnswer(Service):
    def handle(self): # The only method that *has* to be defined. This is the event-triggered entry point.
        self.logger.info('Request Answer: {}'.format(self.request.payload))

       
        response = {}
        q = self.request.payload['question']
        response['question'] = q
        response['answer'] = Simple.missing_answer(q)

        self.logger.info('Response: {}'.format(response))
       
        
        # And return response to the caller
        self.response.payload = response

Overwriting zato_tutorial/my_answer.py


Copy to the hot-load. Then add the channel for the service. Check the logs.

In [3]:
!curl localhost:11223/talk/answer -d '{"question":23}'
!curl localhost:11223/talk/answer -d '{"question":21}'
!curl localhost:11223/talk/answer -d '{"question":22342341}'


{"answer": 19, "question": 23}{"answer": 21, "question": 21}{"answer": -22342299, "question": 22342341}CPU times: user 23 µs, sys: 1 µs, total: 24 µs
Wall time: 62 µs
{"answer": 19, "question": 23}CPU times: user 2.68 ms, sys: 54 µs, total: 2.74 ms
Wall time: 115 ms


In [4]:
import Simple
%time Simple.missing_answer(24)
%time !curl localhost:11223/talk/answer -d '{"question":23}'

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 14.1 µs
{"answer": 19, "question": 23}CPU times: user 3.6 ms, sys: 106 µs, total: 3.71 ms
Wall time: 116 ms
